In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

W0521 23:08:32.024269  8992 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
from tensorflow import keras
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from numpy import array
from keras.preprocessing.text import one_hot

Using TensorFlow backend.


In [3]:
embeddings_index = dict()
f = open('D:\glove.6B\glove.6B.100d.txt', encoding="utf8")
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [4]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']

In [5]:
embeddings_index['the']

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [6]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)

In [7]:
t.index_word

{1: 'work',
 2: 'done',
 3: 'good',
 4: 'effort',
 5: 'poor',
 6: 'well',
 7: 'great',
 8: 'nice',
 9: 'excellent',
 10: 'weak',
 11: 'not',
 12: 'could',
 13: 'have',
 14: 'better'}

In [8]:
t.word_index

{'work': 1,
 'done': 2,
 'good': 3,
 'effort': 4,
 'poor': 5,
 'well': 6,
 'great': 7,
 'nice': 8,
 'excellent': 9,
 'weak': 10,
 'not': 11,
 'could': 12,
 'have': 13,
 'better': 14}

In [9]:
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [10]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [11]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    print(word, i)
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

work 1
done 2
good 3
effort 4
poor 5
well 6
great 7
nice 8
excellent 9
weak 10
not 11
could 12
have 13
better 14


In [12]:
len(embedding_matrix)

15

In [22]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
# model.add(keras.layers.Lambda(lambda x: keras.backend.mean(e, axis=1)))
# model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='sigmoid'))
# # compile the model
model.add(Dense(output_dim=512, activation='softmax'))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# model.compile(loss='binary_crossentropy', optimizer='adagrad', class_mode="binary")

# summarize the model
print(model.summary())

TypeError: The added layer must be an instance of class Layer. Found: <tensorflow.python.keras.layers.core.Lambda object at 0x000002518E294D30>

In [34]:
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False))
# model.add(TimeDistributedMerge(mode='ave'))
model.add(Dense(input_dim=100, output_dim=300, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(input_dim=300, output_dim=300, activation='relu'))
# model.add(Dropout(.5))
# model.add(Dense(input_dim=300, output_dim=300, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(input_dim=300, output_dim=11, activation='sigmoid'))

C:\Users\Aditya\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, activation="relu", units=300)`
  after removing the cwd from sys.path.
C:\Users\Aditya\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=300, activation="relu", units=300)`
  
C:\Users\Aditya\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=300, activation="sigmoid", units=11)`
  # Remove the CWD from sys.path while we load stuff.


In [44]:
# fit the model
model.predict(embedding_matrix[4])

ValueError: Error when checking input: expected embedding_12_input to have shape (4,) but got array with shape (1,)

In [26]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)

In [42]:
embedding_matrix[1]

array([-1.16190001e-01,  4.54470009e-01, -6.92160010e-01,  3.45799997e-02,
        2.63480008e-01, -3.81390005e-01, -2.27899998e-01,  3.72330010e-01,
       -2.05789998e-01,  2.90199995e-01,  1.21140003e-01, -4.27289993e-01,
        5.55729985e-01, -9.42860022e-02, -4.99669999e-01, -2.94779986e-01,
        7.41090000e-01,  2.51910001e-01, -2.74679989e-01,  2.31910005e-01,
        3.82039999e-03,  4.52519991e-02,  2.49699995e-01, -4.15789992e-01,
        3.13069999e-01, -5.84959984e-01, -3.27389985e-01, -6.61889970e-01,
        1.49090007e-01, -2.57710010e-01, -9.48580027e-01,  4.18089986e-01,
       -2.95379996e-01, -4.27110009e-02, -6.99699998e-01,  5.78920007e-01,
       -6.92709982e-02, -3.96329984e-02, -5.64630004e-03, -2.96160012e-01,
       -5.74479997e-01,  1.60099998e-01, -1.06710002e-01,  1.00960001e-01,
       -4.29560006e-01, -2.77850002e-01, -3.00170004e-01, -6.95370018e-01,
        1.79649994e-01, -4.67229992e-01,  1.25110000e-01, -2.90220007e-02,
       -1.59740001e-01,  

In [45]:
l0 = tf.keras.layers.Dense(units=4, input_shape=[300])  
l1 = tf.keras.layers.Dense(units=4)  
l2 = tf.keras.layers.Dense(units=1)  
model = tf.keras.Sequential([l0, l1, l2])

In [48]:
model.compile(loss='crossentropy', optimizer=tf.keras.optimizers.Adam(0.1))
model.fit(celsius_q, fahrenheit_a, epochs=500, verbose=False)
print("Finished training the model")
print(model.predict([100.0]))
print("Model predicts that 100 degrees Celsius is: {} degrees Fahrenheit".format(model.predict([100.0])))
print("These are the l0 variables: {}".format(l0.get_weights()))
print("These are the l1 variables: {}".format(l1.get_weights()))
print("These are the l2 variables: {}".format(l2.get_weights()))

Instructions for updating:
Use tf.cast instead.


W0522 01:52:17.013670  8992 deprecation.py:323] From C:\Users\Aditya\Anaconda3\lib\site-packages\tensorflow\python\ops\math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Finished training the model
[[211.74744]]
Model predicts that 100 degrees Celsius is: [[211.74744]] degrees Fahrenheit
These are the l0 variables: [array([[ 0.01766429, -0.27930024,  0.6918999 , -0.0842316 ]],
      dtype=float32), array([-2.7924762, -3.145586 ,  3.0495803,  2.6537397], dtype=float32)]
These are the l1 variables: [array([[-0.6097427 ,  1.0206141 ,  0.25948015,  0.52247643],
       [ 0.32213086,  0.8575037 ,  1.1987835 ,  0.14238216],
       [ 1.2315762 , -0.8837189 , -0.58433557,  0.13424845],
       [ 1.1857086 , -0.22224444, -0.52304965,  0.48606926]],
      dtype=float32), array([ 3.4458392, -3.3755434, -2.9788616, -1.6323869], dtype=float32)]
These are the l2 variables: [array([[ 0.7611163 ],
       [-0.79602337],
       [-0.9568402 ],
       [-0.31428525]], dtype=float32), array([3.070818], dtype=float32)]


In [47]:
celsius_q    = np.array([-40, -10,  0,  8, 15, 22,  38],  dtype=float)
fahrenheit_a = np.array([-40,  14, 32, 46, 59, 72, 100],  dtype=float)